In [ ]:
!pip install torch
!pip install datasets
!pip install transformers
!pip3 install emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=76f755299edc8395de88c303363180cd40ad994216fd6c9c709b4e3c87d9fb7e
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/247 Project
%ls

/content/drive/.shortcut-targets-by-id/11dCuBITl5umJqjJki52-YAId8zeeW9i7/247 Project
'Christina marg-finetuning-tweetbert.ipynb'          models/
'Claire Christina marg-finetuning-tweetbert.ipynb'   nlpositionality-analysis.ipynb
'Copy of marg-finetuning-tweetbert.ipynb'           'Project Ideas.gdoc'
 data/                                              'Project Proposal.gdoc'
 lda-topic-modeling.ipynb                            test_trainer/
 marg-finetuning-tweetbert.ipynb


In [ ]:
import torch
torch.manual_seed(9)

In [ ]:
!pip install tomotopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 46.5 MB/s eta 0:00:00


In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')

def tokenization(tweet_list):
  tweet_corpus_tokenized = []
  tokenizer = RegexpTokenizer(r'\w+')
  for tweet in tweet_list:
    tokenized_lyric = tokenizer.tokenize(tweet.lower())
    tweet_corpus_tokenized.append(tokenized_lyric)
  return tweet_corpus_tokenized

def token_filtering(tweet_tokenized_list):
  for idx, tweet in enumerate(tweet_tokenized_list):
    filtered_tweets = []
    for token in tweet:
        if len(token) > 2 and not token.isnumeric():
            filtered_tweets.append(token)
    tweet_tokenized_list[idx] = filtered_tweets
  return tweet_tokenized_list

def lemmatization(tweet_corpus_tokenized):
  lemmatizer = WordNetLemmatizer()
  for idx, tweet in enumerate(tweet_corpus_tokenized):
    lemmatized_tokens = []
    for token in tweet:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))
    tweet_corpus_tokenized[idx] = lemmatized_tokens
  return tweet_corpus_tokenized

def remove_stop_words(tweet_corpus_tokenized):
  stop_words = stopwords.words('english')
  for idx, tweet in enumerate(tweet_corpus_tokenized):
    filtered_text = []
    for token in tweet:
        if not contains_non_ascii(token) and token not in stop_words:
            filtered_text.append(token)
    tweet_corpus_tokenized[idx] = filtered_text
  return tweet_corpus_tokenized

def contains_non_ascii(word):
    for char in word:
        if ord(char) >= 128:
            return True
    return False

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import tomotopy as tp
from collections import defaultdict
import random
import pandas as pd

seed = int(random.randint(1, 300000))

# Load the training dataset from CSV
df = pd.read_csv("./data/toxicity_processed_train.csv")

# Extract the text column (the first column)
texts = df.iloc[:, 0].tolist()

tweet_tokenized = tokenization(texts)
tweet_tokenized = token_filtering(tweet_tokenized)
tweet_tokenized = lemmatization(tweet_tokenized)
tweet_tokenized = remove_stop_words(tweet_tokenized)

# Create an LDAModel instance with 3 topics
lda_model = tp.LDAModel(k=3, seed = seed)

print(tweet_tokenized)

# Add documents to the model
for text in tweet_tokenized:
    lda_model.add_doc(text)

# Train the model
lda_model.train(1000)

# Create dictionaries to store documents for each topic
topic_documents = defaultdict(list)

# Assign each document to the topic with the highest probability
for i, text in enumerate(texts):
    doc = lda_model.make_doc(text.split())
    topic_dist = lda_model.infer(doc)
    max_topic = topic_dist[0].argmax()
    topic_documents[max_topic].append(i)  # Store the index of the document

# Split the dataset into three based on the topic assignments
split_datasets = {topic: df.iloc[indices] for topic, indices in topic_documents.items()}

# Print the number of documents in each topic
for topic, documents in split_datasets.items():
    print(f"Topic {topic}: {len(documents)} documents")

[['wonder', 'mads', 'bitter', 'bitch', 'best', 'friend', 'emma', 'healthy', 'happy', 'relationship', 'ha', 'gorgeous', 'baby', 'mads', 'got', 'fucking', 'hermes', 'blanket', 'xxx'], ['evidence', 'pours', 'pothead', 'complete', 'idiot'], ['never', 'heard', 'anyone', 'woman', 'otherwise', 'complain', 'given', 'much', 'freedom', 'hard', 'absolutely', 'ridiculous'], ['fuck', 'country', 'boy'], ['everyone', 'ha', 'every', 'right', 'country', 'decide', 'camp', 'university', 'ground', 'indefinitely', 'use', 'excuse', 'stay'], ['gavin', 'newsom', 'french', 'trannie', 'episode', 'wa', 'aired', 'asked', 'around', 'mate', 'see', 'local', 'outrage', 'matched', 'national', 'outrage', 'heard', 'care', 'saw', 'newsom', 'everyone', 'else', 'doe', 'taking', 'hard', 'fast', 'rule', 'guideline', 'making', 'decision', 'genital', 'mutilating', 'fag'], ['donald', 'sterling', 'former', 'owner', 'los', 'angeles', 'clipper', 'bruce', 'levenson', 'soon', 'former', 'owner', 'atlanta', 'hawk', 'incredibly', 'rich

<ipython-input-90-4c0cdb668713>:29: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  lda_model.train(1000)


Topic 2: 1672 documents
Topic 1: 1839 documents
Topic 0: 1596 documents


In [ ]:
csv_filename_train_0 = "./data/toxicity_processed_train_topic_0.csv"
split_datasets[0].to_csv(csv_filename_train_0, index=False)
csv_filename_train_1 = "./data/toxicity_processed_train_topic_1.csv"
split_datasets[1].to_csv(csv_filename_train_1, index=False)
csv_filename_train_2 = "./data/toxicity_processed_train_topic_2.csv"
split_datasets[2].to_csv(csv_filename_train_2, index=False)

In [ ]:

lda_model.summary()


<Basic Info>
| LDAModel (current version: 0.12.7)
| 5107 docs, 81279 words
| Total Vocabs: 2238, Used Vocabs: 2238
| Entropy of words: 7.07193
| Entropy of term-weighted words: 7.07193
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 1000, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -6.71829
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 3 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 151876 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.00652004 0.0

In [ ]:
lda_model.save('./models/tomotopy-lda-split-{}-{}.bin'.format(lda_model.k, seed), True)

In [ ]:
# Load the test dataset from CSV
test_df = pd.read_csv('./data/toxicity_processed_test.csv')

# Extract the text column (assuming it's the first column, adjust if needed)
test_texts = test_df.iloc[:, 0].tolist()

# Create dictionaries to store documents for each topic in the test dataset
test_topic_documents = defaultdict(list)

# Assign each document in the test dataset to the topic with the highest probability
for i, text in enumerate(test_texts):
    doc = lda_model.make_doc(text.split())
    topic_dist = lda_model.infer(doc)
    max_topic_idx = topic_dist[0].argmax()  # Find the index of the maximum value
    test_topic_documents[max_topic_idx].append(i)  # Store the index of the document

# Load the validation dataset from CSV
val_df = pd.read_csv('./data/toxicity_processed_val.csv')

# Extract the text column (assuming it's the first column, adjust if needed)
val_texts = val_df.iloc[:, 0].tolist()

# Create dictionaries to store documents for each topic in the validation dataset
val_topic_documents = defaultdict(list)

# Assign each document in the validation dataset to the topic with the highest probability
for i, text in enumerate(val_texts):
    doc = lda_model.make_doc(text.split())
    topic_dist = lda_model.infer(doc)
    max_topic_idx = topic_dist[0].argmax()  # Find the index of the maximum value
    val_topic_documents[max_topic_idx].append(i)  # Store the index of the document

# Print the sizes of test and validation datasets for each topic
for topic in range(lda_model.k):
    print(f"Topic {topic}:")
    print(f"  Test dataset size: {len(test_topic_documents[topic])} documents")
    print(f"  Validation dataset size: {len(val_topic_documents[topic])} documents")

Topic 0:
  Test dataset size: 433 documents
  Validation dataset size: 168 documents
Topic 1:
  Test dataset size: 549 documents
  Validation dataset size: 223 documents
Topic 2:
  Test dataset size: 437 documents
  Validation dataset size: 177 documents


In [ ]:
test_datasets_by_topic = {}
val_datasets_by_topic = {}
for topic, indices in test_topic_documents.items():
    test_datasets_by_topic[topic] = test_df.iloc[indices]

# Populate validation dataset by topic
for topic, indices in val_topic_documents.items():
    val_datasets_by_topic[topic] = val_df.iloc[indices]

In [ ]:
print(test_datasets_by_topic[0])

                                                 action  litw  dynahate  \
1     I don't understand at all. Men dressing up as ...   1.0       1.0   
7     Women might wear less revealing athletic cloth...   1.0       1.0   
12    It's outrageous the money involved with these ...   1.0       1.0   
15    being anti-trans is such a weird way to say u ...  -1.0       0.0   
18    The "All Lives Matter" people are in a denial ...   0.0       0.0   
...                                                 ...   ...       ...   
1393  I have never once heard anyone, p@kk1 woman or...   1.0       1.0   
1403  Where did all the good men go? Well im sorry b...   1.0       1.0   
1406  I'm not a racist and I judge everyone individu...   0.0       1.0   
1409  How does it feel to be a genderbender in long-...   1.0       1.0   
1417  nobody brought up Xi Jinping, aka the winner, ...  -1.0       0.0   

      perspective    rewire  hateroberta  gpt4 gender      ethnicity  \
1        0.423492  0.471311

In [ ]:
print(val_topic_documents)

defaultdict(<class 'list'>, {0: [0, 4, 14, 16, 21, 26, 29, 31, 34, 36, 37, 41, 42, 43, 62, 63, 69, 77, 78, 82, 85, 88, 89, 90, 93, 94, 97, 102, 103, 110, 122, 123, 124, 128, 130, 131, 132, 135, 136, 139, 142, 143, 148, 155, 157, 159, 167, 168, 170, 171, 172, 173, 177, 179, 181, 182, 183, 188, 190, 207, 209, 213, 215, 217, 220, 226, 227, 228, 230, 231, 234, 236, 237, 248, 253, 254, 259, 260, 265, 270, 275, 276, 284, 286, 288, 293, 305, 306, 307, 315, 316, 317, 320, 323, 326, 327, 331, 337, 338, 341, 344, 347, 353, 356, 358, 359, 365, 368, 370, 371, 373, 374, 376, 382, 390, 399, 401, 402, 403, 406, 412, 426, 430, 432, 435, 442, 445, 447, 448, 452, 455, 464, 466, 468, 469, 471, 474, 479, 480, 482, 492, 495, 498, 499, 501, 504, 507, 508, 509, 513, 518, 519, 520, 521, 526, 528, 529, 534, 548, 549, 550, 552, 556, 558, 559, 561, 566, 567], 2: [1, 8, 12, 17, 20, 22, 23, 24, 28, 30, 46, 52, 54, 56, 58, 60, 61, 64, 68, 70, 79, 98, 100, 104, 107, 111, 112, 113, 114, 116, 117, 120, 126, 127, 133, 

In [ ]:
csv_filename_test_0 = "./data/toxicity_processed_test_topic_0.csv"
test_datasets_by_topic[0].to_csv(csv_filename_test_0, index=False)
csv_filename_test_1 = "./data/toxicity_processed_test_topic_1.csv"
test_datasets_by_topic[1].to_csv(csv_filename_test_1, index=False)
csv_filename_test_2 = "./data/toxicity_processed_test_topic_2.csv"
test_datasets_by_topic[2].to_csv(csv_filename_test_2, index=False)

csv_filename_val_0 = "./data/toxicity_processed_val_topic_0.csv"
val_datasets_by_topic[0].to_csv(csv_filename_val_0, index=False)
csv_filename_val_1 = "./data/toxicity_processed_val_topic_1.csv"
val_datasets_by_topic[1].to_csv(csv_filename_val_1, index=False)
csv_filename_val_2 = "./data/toxicity_processed_val_topic_2.csv"
val_datasets_by_topic[2].to_csv(csv_filename_val_2, index=False)